In [1]:
import pandas as pd
import datetime

In [2]:
data = pd.read_excel('СПУ по пациентам НСК.xlsx')

In [6]:
data = data.sort_values('Дата начала СПУ из карточки Пациента').reset_index(drop=True)

In [7]:
group_pacients = data.groupby('Пациент', as_index=False).agg({'СПУ из карточки Пациента':list, 'Дата начала СПУ из карточки Пациента':list})

In [18]:
group_pacients['Кол-во СПУ'] = group_pacients['СПУ из карточки Пациента'].apply(lambda x: len(set(x)))

In [19]:
group_pacients_new = group_pacients[group_pacients['Кол-во СПУ'] > 1]

In [21]:
group_pacients_new = group_pacients_new.reset_index(drop=True)

In [32]:
temp_dct = {}

for i in group_pacients_new.index:
    temp_dct.setdefault(group_pacients_new['Пациент'][i], {})
    for j in range(len(group_pacients_new['СПУ из карточки Пациента'][i]) - 1):
        if group_pacients_new['СПУ из карточки Пациента'][i][j] == group_pacients_new['СПУ из карточки Пациента'][i][j+1]:
            continue
        else:
            temp_dct[group_pacients_new['Пациент'][i]].setdefault(group_pacients_new['СПУ из карточки Пациента'][i][j], group_pacients_new['Дата начала СПУ из карточки Пациента'][i][j].date())
    temp_dct[group_pacients_new['Пациент'][i]].setdefault(group_pacients_new['СПУ из карточки Пациента'][i][len(group_pacients_new['СПУ из карточки Пациента'][i]) - 1], group_pacients_new['Дата начала СПУ из карточки Пациента'][i][len(group_pacients_new['СПУ из карточки Пациента'][i]) - 1].date())        

In [40]:
df_dict = {'name':[], 'spu':[], 'date':[]}

for key, val in temp_dct.items():
    for key1, val1 in val.items():
        df_dict['name'].append(key)
        df_dict['spu'].append(key1)
        df_dict['date'].append(val1)
    df_dict['name'].append('')
    df_dict['spu'].append('')
    df_dict['date'].append('')    

In [41]:
data_new_group = pd.DataFrame(df_dict)

In [42]:
data_new_group

,name,spu,date
0,Абдулова Вероника Евгеньевна,Скидка 5%,2017-12-03
1,Абдулова Вероника Евгеньевна,Скидка 15%,2020-01-30
2,,,
3,Абрамова Татьяна Викторовна,Скидка 5%,2017-11-15
4,Абрамова Татьяна Викторовна,Скидка 15%,2018-12-07
...,...,...,...
578,Яковлев Кирилл Владимирович,Скидка 15%,2018-12-07
579,,,
580,Ястрембская Елена Юрьевна,Скидка 5%,2019-11-21
581,Ястрембская Елена Юрьевна,Скидка 7%,2020-02-05


In [44]:
data_new_group.to_excel('Изменение СПУ.xlsx', index=False)